In [1]:
import numpy as np
import pandas as pd

In [2]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';', dtype={'ISBN': str, 'Title': str, 'Author': str, 'Year': np.int16, 'Publisher': str})
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';', dtype={'User-ID': np.int32, 'ISBN': str, 'Rating': np.int8})

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_2623/2231409931.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [8]:
import sys
sys.path.append('../..')

from src.evaluation import average_precision
from sklearn.model_selection import train_test_split

In [16]:
import re

books_data=books.merge(ratings,on="ISBN")

df=books_data.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
# df.drop(columns=["ISBN","Year"],axis=1,inplace=True)
df.drop(index=df[df["Rating"]==0].index,inplace=True)
df["Title"]=df["Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())

# df.rename(columns={"Title":"Book-Title", "Author":"Book-Author", "Rating":"Book-Rating"},inplace=True)



<>:10: SyntaxWarning: invalid escape sequence '\W'
<>:10: SyntaxWarning: invalid escape sequence '\W'
/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_2623/1543249917.py:10: SyntaxWarning: invalid escape sequence '\W'
  df["Title"]=df["Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())


In [17]:
df.head(5)

,ISBN,Title,Author,Year,Publisher,User-ID,Rating
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,67544,8
8,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,116866,9
9,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,123629,9


In [25]:
rating_count=pd.DataFrame(df["Title"].value_counts())
rare_books=rating_count[rating_count["count"]<=20].index
common_books=df[~df["Title"].isin(rare_books)]

common_books.head()
ratings = common_books[['ISBN', 'User-ID', 'Rating']]


In [26]:
train_ratings, test_ratings = train_test_split(ratings, test_size=0.3, random_state=42)

In [27]:
import src.recommenders.item_collaborative_filtering as icf

model = icf.ItemRecommender()
model.fit(users=users, items=books, ratings=train_ratings)

/Users/oleh.sannikov/Documents/Навчання/Науки про дані/RecSys/CP/rs-ucu-books-recommender/experiments/sunnycows/../../src/utils/data_preprocessing.py:81: RuntimeWarning: Mean of empty slice.
  means = np.array([normalized[i].data.mean() for i in range(normalized.shape[0])])
/Users/oleh.sannikov/miniconda3/envs/recsys/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [28]:
test_users = test_ratings['User-ID'].unique()[:1000]
len(test_users)

1000

In [29]:
books.shape

(271379, 5)

In [32]:
best_items = model.predict(test_users, items=books)

In [31]:
len(best_items)

1000

In [35]:
best_items

{215180: Empty DataFrame
 Columns: [index, ISBN, Title, Author, Year, Publisher]
 Index: [],
 95359:      index        ISBN                               Title        Author  \
 36      36  0375406328                         Lying Awake  Mark Salzman   
 107    107  0786868716  The Five People You Meet in Heaven   Mitch Albom   
 
      Year        Publisher  
 36   2000  Alfred A. Knopf  
 107  2003         Hyperion  ,
 264061: Empty DataFrame
 Columns: [index, ISBN, Title, Author, Year, Publisher]
 Index: [],
 29855: Empty DataFrame
 Columns: [index, ISBN, Title, Author, Year, Publisher]
 Index: [],
 127200: Empty DataFrame
 Columns: [index, ISBN, Title, Author, Year, Publisher]
 Index: [],
 39441: Empty DataFrame
 Columns: [index, ISBN, Title, Author, Year, Publisher]
 Index: [],
 76626:       index        ISBN                                              Title  \
 121     121  0743439740  Every Breath You Take : A True Story of Obsess...   
 1451   1451  0385508417                 

In [33]:
from tqdm import tqdm

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    user_id = test_users[i]
    ap = average_precision(best_items[user_id]['ISBN'].to_numpy(), relevant_items['ISBN'].to_numpy(), k=5)
    # print(f"Relevant items: {relevant_items['ISBN'].to_numpy(), best_items[user_id]}")
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 1000/1000 [00:00<00:00, 1009.62it/s]


0.0002666666666666667